In [1]:
import sys, os
from pymongo import MongoClient

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

# https://stackoverflow.com/questions/41323423/plotly-inside-jupyter-notebook-python
init_notebook_mode(connected=True) # initiate notebook for offline plot

import pandas as pd

In [2]:
sys.path.append("..")
import config # ../config.py
import common # ../common.py

# run on Antec
config.mongodb_ip = "192.168.1.140"

client = MongoClient(host=config.mongodb_ip, port=config.mongodb_port)
db = client['steam']

## Failed appids excluded from future attempts or analysis

In [3]:
failed_appids_df = pd.DataFrame(list(db['apps'].find({"failureCount": {"$gte":3}}, {"appid":1,"name":1,"_id":False})))
failed_appids_df

,appid,name
0,3402,Hammer Heads Deluxe Demo
1,3442,Rocket Mania Deluxe Demo
2,5803,Darwinia_ESRB
3,12630,Legend: Hand of God
4,15310,The Settlers: Heritage of Kings
5,19930,The Settlers: Rise of an Empire Gold Edition
6,23330,The Last Remnant Demo
7,33220,Tom Clancy's Splinter Cell: Conviction
8,33910,Arma 2
9,34000,Football Manager 2010


In [4]:
grouped_df = db['apps'].aggregate([
{"$match":
    {"$or": [{"failureCount": {"$lt": 3}}, {"failureCount": {"$exists": False}}]}
},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$updated_date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'updated_date'}, inplace=True)
grouped_df

,updated_date,count
0,2020-07-21,1915
1,2020-07-22,3661
2,2020-07-23,3480
3,2020-07-24,3347
4,2020-07-25,3946
5,2020-07-26,3481
6,2020-07-27,3489
7,2020-07-28,3408
8,2020-07-29,3730
9,2020-07-30,3513


## Large numbers of old AppID entries means `refreshsteam.py` needs to be run.  This excludes consistently failed appids.

In [5]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['updated_date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='AppIDs By Last Updated Date',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [6]:
grouped_df = db['pricehistory'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2018-09-04,8215
1,2018-09-05,43441
2,2018-09-06,37421
3,2018-09-07,23655
4,2018-09-08,37188
5,2018-09-10,2963
6,2018-09-11,45999
7,2018-09-12,20210
8,2018-09-13,269
9,2018-09-14,37390


## This is historical data from the `pricehistory` Mongo collection.  This is updated via the `updatepricehistory.py` script.

In [7]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Price History',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [8]:
grouped_df = db['steamusers'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2018-12-19,119
1,2018-12-20,340
2,2018-12-21,751
3,2018-12-22,1021
4,2018-12-23,932
5,2018-12-24,980
6,2018-12-25,938
7,2018-12-26,924
8,2018-12-27,1106
9,2018-12-28,1287


## These are the number of records we have per day for the `steamusers` Mongo collection.  This is created using the `steamusers.py` script.

In [9]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Records Per Day for steamusers',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [10]:
grouped_df = db['topgames'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,2019-01-08,300
1,2019-01-09,2300
2,2019-01-10,100
3,2019-01-13,4000
4,2019-01-14,9000
5,2019-01-15,8400
6,2019-01-16,9200
7,2019-01-17,8500
8,2019-01-18,9100
9,2019-01-19,5200


## These are the number of entries we have per day for the `topgames` Mongo collection.  Each query returns the current top 100 games being played.  This is run every 15 minutes.

In [11]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Records Per Day for topgames',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [12]:
grouped_df = db['apps'].aggregate([
{"$match": {"type": {"$in": ["game", "dlc"]}}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$reviews.last_updated"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,count
0,None,70600
1,2020-07-26,5
2,2020-07-27,3
3,2020-07-28,2
4,2020-08-01,50
5,2020-08-02,2
6,2020-08-03,5
7,2020-08-04,719
8,2020-08-05,1246
9,2020-08-06,1409


## These are the number of Steam games where we have review information from Steam and when they were last updated

In [13]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['count']
    )]

layout = Layout(
        title='Distribution of Steam game reviews per day',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [14]:
steam_grouped_df = db['opencritic'].aggregate([
{"$match": {"steamId": {"$exists": True, "$ne": None}}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

steam_grouped_df = pd.DataFrame(list(steam_grouped_df))
steam_grouped_df.rename(columns={steam_grouped_df.columns[0] : 'date'}, inplace=True)

nonsteam_grouped_df = db['opencritic'].aggregate([
{"$match": {"$or": [{"steamId": {"$exists": False}}, {"steamId": None}]}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

nonsteam_grouped_df = pd.DataFrame(list(nonsteam_grouped_df))
nonsteam_grouped_df.rename(columns={nonsteam_grouped_df.columns[0] : 'date'}, inplace=True)

print(steam_grouped_df)
print(nonsteam_grouped_df)

          date  count
0   2020-07-30    175
1   2020-07-31    304
2   2020-08-01    285
3   2020-08-02    294
4   2020-08-03    275
5   2020-08-04    303
6   2020-08-05    310
7   2020-08-06    292
8   2020-08-07    281
9   2020-08-08    256
10  2020-08-09    317
11  2020-08-10    322
12  2020-08-11    326
13  2020-08-12    284
14  2020-08-13    296
15  2020-08-14    327
16  2020-08-15    164
          date  count
0   2020-07-29      1
1   2020-07-30    159
2   2020-07-31    256
3   2020-08-01    272
4   2020-08-02    267
5   2020-08-03    202
6   2020-08-04    253
7   2020-08-05    220
8   2020-08-06    259
9   2020-08-07    274
10  2020-08-08    303
11  2020-08-09    237
12  2020-08-10    231
13  2020-08-11    228
14  2020-08-12    280
15  2020-08-13    271
16  2020-08-14    237
17  2020-08-15    155


## This is the breakdown of OpenCritic data we have by last accessed/updated date.

In [15]:
# https://plot.ly/python/bar-charts/#stacked-bar-chart

steamTrace = Bar(
    x=steam_grouped_df['date'],
    y=steam_grouped_df['count'],
    name='Games w/steamId'
)
nonSteamTrace = Bar(
    x=nonsteam_grouped_df['date'],
    y=nonsteam_grouped_df['count'],
    name='Games without steamId'
)

data = [steamTrace, nonSteamTrace]

layout = Layout(
        title='Distribution of OpenCritic Records Per Day Grouped By steamId',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        ),
        barmode='stack'
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

## This is counts of Twitch records by day grouped by a tie to Steam

In [16]:
steam_grouped_df = db['twitchhistorical'].aggregate([
{"$match": {"steamId": {"$exists": True}}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

steam_grouped_df = pd.DataFrame(list(steam_grouped_df))
steam_grouped_df.rename(columns={steam_grouped_df.columns[0] : 'date'}, inplace=True)

nonsteam_grouped_df = db['twitchhistorical'].aggregate([
{"$match": {"steamId": {"$exists": False}}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "count" : { "$sum" : 1} } },
{"$sort": {"_id": 1}}
])

nonsteam_grouped_df = pd.DataFrame(list(nonsteam_grouped_df))
nonsteam_grouped_df.rename(columns={nonsteam_grouped_df.columns[0] : 'date'}, inplace=True)

# https://plot.ly/python/bar-charts/#stacked-bar-chart

steamTrace = Bar(
    x=steam_grouped_df['date'],
    y=steam_grouped_df['count'],
    name='Games w/steamId'
)
nonSteamTrace = Bar(
    x=nonsteam_grouped_df['date'],
    y=nonsteam_grouped_df['count'],
    name='Games without steamId'
)

data = [steamTrace, nonSteamTrace]

layout = Layout(
        title='Distribution of Twitch Records Per Day Grouped By steamId',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Records Count'
        ),
        barmode='stack'
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [21]:
grouped_df = db['bandwidth'].aggregate([
{"$match": {}},
{"$group":  {"_id": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}},
             "bytes" : { "$sum" : "$bytes"} } },
{"$sort": {"_id": 1}}
])

grouped_df = pd.DataFrame(list(grouped_df))
grouped_df.rename(columns={grouped_df.columns[0] : 'date'}, inplace=True)
grouped_df

,date,bytes
0,2020-08-24,162778


## Bandwidth used

In [22]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grouped_df['date'],
        y=grouped_df['bytes']
    )]

layout = Layout(
        title='Data downloaded per day',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Bytes'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

## Storage and disk usage information

In [19]:
for collection in db.list_collection_names():
    stats = db.command("collstats", str(collection))
    print(collection + " size: " + common.sizeof_fmt(stats['size']))

print("--------------------")
stats = db.command("dbstats")
print("Total Database Size: " + common.sizeof_fmt(stats['dataSize']))

opencritic size: 124.0 MiB
apps size: 746.6 MiB
bandwidth size: 94.0 B
twitchhistorical size: 2.7 GiB
pricehistory size: 1.3 GiB
steamusers size: 26.6 MiB
topgames size: 788.2 MiB
--------------------
Total Database Size: 5.7 GiB


## Check index size across collections

In [20]:
total_index = 0
for collection in db.list_collection_names():
    stats = db.command("collstats", str(collection))
    total_index = total_index + stats['totalIndexSize']
    print(collection + " size: " + common.sizeof_fmt(stats['totalIndexSize']))

print("--------------------")
print("Total Index Size: " + common.sizeof_fmt(total_index))

opencritic size: 468.0 KiB
apps size: 5.5 MiB
bandwidth size: 72.0 KiB
twitchhistorical size: 252.6 MiB
pricehistory size: 410.8 MiB
steamusers size: 11.5 MiB
topgames size: 86.4 MiB
--------------------
Total Index Size: 767.3 MiB
